In [29]:
import numpy as np
import netCDF4 as nc
import glob
import matplotlib.pyplot as plt
import re
import pandas as pd
from datetime import datetime, timedelta
import os

In [4]:
file_path = '/gws/nopw/j04/tone_ico_gws/cloudnet/troll/radar/'

file_list = sorted(glob.glob('/gws/nopw/j04/tone_ico_gws/cloudnet/troll/radar/2025*.nc'))  # Adjust path based on month


In [22]:
ds = nc.Dataset(file_list[1])

In [7]:
print(ds)

<class 'netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    Conventions: CF-1.8
    cloudnetpy_version: 1.70.1
    file_uuid: 07b14537-223d-4d02-90be-88d618250ed2
    cloudnet_file_type: radar
    title: RPG-FMCW-35 cloud radar from Troll Station
    year: 2025
    month: 03
    day: 01
    location: Troll Station
    history: 2025-03-02 00:35:51 +00:00 - radar file created
    source: RPG-Radiometer Physics RPG-FMCW-35
    references: https://doi.org/10.21105/joss.02123
    pid: https://hdl.handle.net/21.12132/1.07b14537223d4d02
    cloudnet_processing_version: 2.51.2
    instrument_pid: https://hdl.handle.net/21.12132/3.82bd078f71da4bba
    dimensions(sizes): time(14123), range(710), chirp_sequence(3)
    variables(dimensions): int32 file_code(), int32 program_number(), int32 model_number(), float32 radar_frequency(), float32 antenna_separation(), float32 antenna_diameter(), float32 antenna_gain(), float32 half_power_beam_width(), int32 dual_polarization(

In [23]:
#print(ds.variables['Zh'])
print(ds.variables['time'])
print(ds.variables['time'][1]-ds.variables['time'][0])
print(ds.variables['time'][-3]-ds.variables['time'][-2])

<class 'netCDF4.Variable'>
float32 time(time)
    units: hours since 2025-03-02 00:00:00 +00:00
    long_name: Time UTC
    standard_name: time
    axis: T
    calendar: standard
unlimited dimensions: 
current shape = (13536,)
filling off
0.00081944466
-0.00082206726


In [ ]:
radar_data = pd.DataFrame()

for file in file_list:
    dset = nc.Dataset(file)

    dbz = np.array(dset.variables['Zh'][:, 0])
    dbz[dbz > 1000] = np.nan
    z = 10 ** (dbz / 10)

    a = 18
    b = 1.1

    precip = (z / a) ** (1 / b)

    basename = os.path.basename(file)  # remove directory path
    date_str = basename.split('_')[0]  # get "YYYYMMDD"
    base_time = datetime.strptime(date_str, '%Y%m%d')

    time_var = ds.variables['time'][:]  # shape (n,)
    timestamps = [base_time + timedelta(hours=float(h)) for h in time_var]
    

    df = pd.DataFrame({'snowfall': precip}, index=pd.to_datetime(timestamps))
    all_data = pd.concat([all_data, df])


